In [4]:
# Reference from "https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html"
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("ggplot")
np.random.seed(1000)

from IPython.display import Image
from numpy.core.umath_tests import matrix_multiply as mm

from scipy.optimize import minimize
from scipy.stats import bernoulli, binom

In [6]:
# Define the function which calculates the log-likelihood of the data sample
def neg_loglik(thetas, n, xs, zs):
    likelihood = -np.sum([binom(n, thetas[z]).logpmf(x) for (x, z) in zip(xs, zs)])
    return likelihood

In [18]:
m = 10
theta_A = 0.8
theta_B = 0.3
thetas = [theta_A, theta_B]

coin_A = bernoulli(theta_A)
coin_B = bernoulli(theta_B)

xs = map(sum, [coin_A.rvs(m), coin_A.rvs(m), coin_B.rvs(m), coin_A.rvs(m), coin_B.rvs(m)])
zs = [0, 0, 1, 0, 1]
xs = np.array(list(xs))
xs

array([7, 8, 5, 7, 2])

In [25]:
ML_A = np.sum(xs[[0, 1, 3]]) / (3 * m)
ML_B = np.sum(xs[[2, 4]]) / (2 * m)
print("The correct MLE is: MLA {} MLB {}".format(ML_A, ML_B))

The correct MLE is: MLA 0.7333333333333333 MLB 0.35


In [27]:
# Optimize using the neg_loglik function defined above
minimize(neg_loglik, 
         [0.5, 0.5], 
         (m, xs, zs), 
         bounds = [(0, 1), (0, 1)], 
         method = "TNC",
         options = {"maxiter": 100}) # This minimization uses truncated Newton Alogorithm


     fun: 7.62865037343645
     jac: array([ 0.00042597, -0.00062537])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 34
     nit: 8
  status: 1
 success: True
       x: array([0.73333611, 0.34999288])

What if we don't know which coin it is? The actual Expectation Maximization Algorithm comes!

In [29]:
xs = np.array([(5, 5), (9, 1), (8, 2), (4, 6), (7, 3)])
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])
# The criterion for convergence
tol = 0.01
max_iter = 100
likelihood = 0

for i in range(max_iter):
    ws_A = []
    ws_B = []
    vs_A = []
    vs_B = []
    likelihood_ = 0

    # E-Step: Calculate probability distributions over possible completions
    for x in xs:
        E_A = np.sum([x * np.log(thetas[0])])
        E_B = np.sum([x * np.log(thetas[1])])

        # Use softmax function for calculating the weights
        w_A = np.exp(E_A) / (np.exp(E_A) + np.exp(E_B))
        w_B = np.exp(E_B) / (np.exp(E_B) + np.exp(E_B))

        ws_A.append(w_A)
        ws_B.append(w_B)

        vs_A.append(np.dot(w_A, x))
        vs_B.append(np.dot(w_B, x))

        # Compute the updated likelihood
        likelihood_ = w_A * E_A + w_B * E_B
    
    # M-Step: Update parameters given current distribution
    thetas[0] = np.sum(vs_A, 0) / np.sum(vs_A)
    thetas[1] = np.sum(vs_B, 0) / np.sum(vs_B)
    
    print("Iteration i: {}".format(i))
    print("theta_A: {}, theta_B: {}, likelihood: {}".format(thetas[0], thetas[1], likelihood_))
    
    # If the update difference is less than a threshold, stop the loop
    if (np.abs(likelihood - likelihood_) < tol):
        break
    likelihood = likelihood_

Iteration i: 0
theta_A: [0.71301224 0.28698776], theta_B: [0.66 0.34], likelihood: -7.5591459965110435
Iteration i: 1
theta_A: [0.70259325 0.29740675], theta_B: [0.66 0.34], likelihood: -6.178236132586252
Iteration i: 2
theta_A: [0.6938765 0.3061235], theta_B: [0.66 0.34], likelihood: -6.182250042901686
